In [ ]:
from nixtla import NixtlaClient
import pandas as pd
nixtla_client = NixtlaClient(
    api_key='nixak-NUbao7AfDoqbzMPUCOBd0OJbUGsODIvE2iukG1MZAHH6UqbJ07Ea5CoUOIoKZ0pn50KDcwaWaBy0M9IH'
) #this is my API you can change it.
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :)


True

In [ ]:
import pandas as pd

solar_df = pd.read_csv("pv_8kW_5minresample_concat_impute.csv", parse_dates=["Datetime"], index_col="Datetime")
#solar_df = pd.read_csv("load_feature_5minresample_edited.csv", parse_dates=["Datetime"], index_col="Datetime")


In [ ]:
import pandas as pd
#with cloud

target_col = 'Irradiance (W/m2)_imputed'
#target_col = 'Ltot_impute (kW)'
lag_feature = ['Iclr_imputed', 'Inwp_imputed', 'Tnwp_imputed']
#lag_feature = ['Tnwp' , 'lab_hour','lab_day_machine','lab_day_power','is_work_day']
future_exogs_cols = ['Iclr_imputed', 'Inwp_imputed', 'Tnwp_imputed']
#future_exogs_cols = ['Tnwp' , 'lab_hour','lab_day_machine','lab_day_power','is_work_day']

start_date = pd.Timestamp("2024-01-01")
end_date = pd.Timestamp("2024-12-28")

current_date = start_date
forecast_results = []  # เก็บผล forecast แต่ละวัน

while current_date <= end_date:
    # 1️⃣ historical 1 วัน
    historical_range = pd.date_range(
        start=current_date, # lag 1,3,7 เเก้ตรงนี้
        end=current_date + pd.Timedelta(hours=23, minutes=55),
        freq="5min"
    )

    # 2️⃣ future 3 วัน
    future_start = current_date + pd.Timedelta(days=1)
    future_end = current_date + pd.Timedelta(days=3, hours=23, minutes=55)
    future_range = pd.date_range(start=future_start, end=future_end, freq="5min")
    
    n_forecasts = len(future_range)  # จำนวนจุดที่จะ forecast

    # 3️⃣ เตรียม DataFrame
    sliced_solar_df = (
        solar_df.loc[historical_range]
        .reset_index()
        .rename(columns={'index': 'Datetime'})[['Datetime', target_col] + lag_feature]
    )

    sliced_solar_future_exog = (
        solar_df.loc[future_range]
        .reset_index()
        .rename(columns={'index': 'Datetime'})[['Datetime'] + future_exogs_cols]
    )

    # 4️⃣ forecast ด้วย TimeGPT
    timegpt_solar_result = nixtla_client.forecast(
        df=sliced_solar_df,
        h=n_forecasts,
        X_df=sliced_solar_future_exog,
        freq='5min',
        time_col='Datetime',
        target_col=target_col,
    )

    # # # 5️⃣ เก็บผลลัพธ์พร้อมวันที่อ้างอิง
    forecast_results.append({
        'base_date': current_date,
        'forecast': timegpt_solar_result
    })

    # 6️⃣ ไปวันถัดไป
    current_date += pd.Timedelta(days=1)
    # current_date += pd.Timedelta(minutes= 5 ) ถ้า จะเอา roll 5 mins

# print(f"สร้าง forecast ครบ {len(forecast_results)} วัน (1 วันต่อรอบ)")
    

INFO:nixtla.nixtla_client:Validating inputs...
c:\Users\petch\ana\Lib\site-packages\nixtla\nixtla_client.py:394: UserWarning: `df` contains the following exogenous features: ['CI_CM_imputed', 'CI_RGB_imputed'], but they were not found in `X_df` nor declared in `hist_exog_list`. They will be ignored.
  warnings.warn(
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...
INFO:nixtla.nixtla_client:Using future exogenous features: ['Iclr_imputed', 'Inwp_imputed', 'Tnwp_imputed']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
c:\Users\petch\ana\Lib\site-packages\nixtla\nixtla_client.py:394: UserWarning: `df` contains the following exogenous features: ['CI_CM_imputed', 'CI_RGB_imputed'], but they were not found in `X_df` nor declared in `hist_exog_list`. They will be ignored.
  warnings.warn(
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future e

In [ ]:
st = pd.Timestamp("2024-01-01") #เเก้ไขได้
ed = pd.Timestamp("2024-01-02") #เเก้ไขได้
cur_date = st
count = 1
diction = dict()
while cur_date <= ed:
    if count > 363:
        break
    select_for = None
    for t in forecast_results:
        if t['base_date'] == cur_date:
            select_for = t['forecast']
            break
    period = pd.date_range(
        start=cur_date,
        end=cur_date + pd.Timedelta(hours=23, minutes=55),
        freq="5min"
    )
    
    period_df = (
        solar_df.loc[period]
        .reset_index()
        .rename(columns={'index': 'Datetime'})[['Datetime', target_col] + future_exogs_cols]
    )
    key = period_df.tail(1)["Datetime"].iloc[0]  # ใช้ Timestamp เดี่ยว
    diction[key] = forecast_results[count-1]['forecast'].values  # .values() เป็น numpy array

    cur_date += pd.Timedelta(days=1)
    count += 1
    last_row_of_day = period_df.tail(1)
    


In [ ]:
import pandas as pd
import numpy as np



execution = pd.DataFrame()

for key, arr in diction.items():
    # ดึงค่าพยากรณ์
    values = [v for _, v in arr]
    horizon_cols = [f"I_hat(t+{i+1}|t)" for i in range(len(values))]
    #horizon_cols = [f"Load_hat(t+{i+1}|t)" for i in range(len(values))] #สำหรับ Load

    # สร้าง DataFrame แถวเดียว
    temp_df = pd.DataFrame([values], columns=horizon_cols) #สร้าง col เปล่าตามตัวเเปรใน horizon

    # --- insert Datetime และคอลัมน์เปล่า I(t) ---
    temp_df.insert(0, "I(t)", np.nan)   # เพิ่มคอลัมน์เปล่า I(t)
    #temp_df.insert(0, "Load(t)", np.nan) #columns Load(t)
    temp_df.insert(0, "Datetime", key)  

    # concat
    execution = pd.concat([execution, temp_df], ignore_index=True)

# เรียงตาม Datetime
execution = execution.sort_values("Datetime").reset_index(drop=True)

In [ ]:
old_data = pd.read_csv("pv_8kW_5minresample_concat_impute.csv") # เอามา merge รวมกับ dataframe
#old_data = pd.read_csv("JSS_proj\load_feature_5minresample_edited.csv")
old_data["Datetime"] = pd.to_datetime(old_data["Datetime"])
# Merge เพื่อนำค่า I(t)
execution = execution.merge(
    old_data[['Datetime', 'Irradiance (W/m2)_imputed']],
    on='Datetime',
    how='left'
)
# execution = execution.merge(                      #สำหรับ Load
#     old_data[['Datetime', 'Ltot_impute (kW)']],
#     on='Datetime',
#     how='left'
# )

execution['I(t)'] = execution['Irradiance (W/m2)_imputed']
#execution["Load(t)"] = execution['Ltot_impute (kW)']

execution = execution.drop(columns=['Irradiance (W/m2)_imputed'])
# execution = execution.drop(columns=['Ltot_impute (kW)'])

In [ ]:
#execution.to_csv("TimeGPT_lag1day(new_ver).csv") #สร้างไฟล์ base on execution time

In [ ]:
import pandas as pd
import numpy as np

#Convert for Solar
STEP_MIN = 5  # 1 step = 5 นาที


input_data_converse = pd.read_csv("TimeGPTlag7daySolar.csv", parse_dates=["Datetime"])
e = pd.read_csv("pv_8kW_5minresample_concat_impute.csv")
e["Datetime"] = pd.to_datetime(e["Datetime"])

# -------------------------
# PARSE HORIZON COLUMNS
# -------------------------
column_frame = [c for c in input_data_converse.columns if c.startswith("I_hat(t+")]
# map: column name -> k (เช่น "I_hat(t+73|t)" -> 73)
col2k = pd.Series(column_frame, index=column_frame).str.extract(r"I_hat\(t\+(\d+)\|t\)")[0].astype(int).to_dict()

# -------------------------
# LONG TABLE: (origin -> target)
# -------------------------
long = input_data_converse.melt(
    id_vars=["Datetime", "I(t)"], value_vars=column_frame,
    var_name="col", value_name="yhat"
)
long["k"] = long["col"].map(col2k).astype(int)
long["Datetime_target"] = long["Datetime"] + pd.to_timedelta(long["k"] * STEP_MIN, unit="min")
long = long.rename(columns={"Datetime":"Datetime_origin", "I(t)":"y_origin"})
long = long[["Datetime_origin", "Datetime_target", "k", "yhat"]]

# -------------------------
# WIDE (by k): yhat(t|t-k)  for k=1..864
# หมายเหตุ: ในเคส origin วันละครั้งที่ 23:55 ตารางนี้จะค่อนข้าง "sparse"
# -------------------------
wide_k = long.pivot_table(index="Datetime_target", columns="k", values="yhat", aggfunc="last")
max_k = max(col2k.values())
# ให้มีครบทุกคอลัมน์ 1..max_k
wide_k = wide_k.reindex(columns=range(1, max_k+1))
wide_k.columns = [f"yhat(t|t-{k})" for k in wide_k.columns]

# แนบค่าจริง y(t) ถ้ามีที่เวลา target (ถ้าไม่มี จะเป็น NaN)
# ---- ช่วยประหยัดหน่วยความจำ ----
wide_k = wide_k.sort_index()          # เรียงด้วย index (Datetime_target) แทน sort_values ทีหลัง
wide_k = wide_k.astype("float32")     # ลด dtype จาก float64 -> float32

# เปลี่ยนชื่อคอลัมน์ yhat(...) -> Ihat(...)
wide_k.columns = [c.replace("yhat(", "Ihat(") for c in wide_k.columns]

# ค่าจริงที่เวลา target
actual_at_target = (
    input_data_converse[["Datetime", "I(t)"]]
      .rename(columns={"Datetime": "Datetime_target"})
      .set_index("Datetime_target")
      .sort_index()
)
if actual_at_target["I(t)"].dtype != "float32":
    actual_at_target["I(t)"] = actual_at_target["I(t)"].astype("float32")

# join แทน merge + sort_values (ประหยัด RAM มากกว่า)
eval_wide = wide_k.join(actual_at_target[["I(t)"]], how="outer")

# รีเซ็ต index และจัดคอลัมน์ตามต้องการ
eval_wide.index.name = "Datetime"
eval_wide = eval_wide.reset_index()

# ให้มี Ihat(t|t-1)..Ihat(t|t-max_k) ครบและเรียง
max_k = max(col2k.values())
need_column = [f"Ihat(t|t-{k})" for k in range(1, max_k+1)]
for name in need_column:
    if name not in eval_wide.columns:
        eval_wide[name] = np.nan

eval_wide = eval_wide[["Datetime", "I(t)"] + need_column]


# -------------------------
# OPTIONAL: version แยกตาม "วันของ origin"
# (เหมาะมากสำหรับเคส forecast วันละครั้ง 23:55)
# จะได้คอลัมน์: yhat_from_origin_D-0, yhat_from_origin_D-1, yhat_from_origin_D-2, yhat_from_origin_D-3
# -------------------------
tmp = long.copy()
tmp["origin_day_offset"] = (tmp["Datetime_target"].dt.normalize() - tmp["Datetime_origin"].dt.normalize()).dt.days
format_to_cal = tmp.pivot_table(index="Datetime_target", columns="origin_day_offset", values="yhat", aggfunc="last")

# จัดชื่อคอลัมน์ให้อ่านง่าย
if format_to_cal is not None and format_to_cal.shape[1] > 0:
    new_cols = {}
    for off in format_to_cal.columns:
        if off == 0:
            new_cols[off] = "yhat_from_origin_D-0"   # พยากรณ์ที่ทำเมื่อคืน 23:55 ของ "วันเดียวกัน"
        elif off == 1:
            new_cols[off] = "yhat_from_origin_D-1"   # ทำที่ 23:55 ของ "เมื่อวาน"
        elif off == 2:
            new_cols[off] = "yhat_from_origin_D-2"
        elif off == 3:
            new_cols[off] = "yhat_from_origin_D-3"
        else:
            new_cols[off] = f"yhat_from_origin_D-{off}"
    format_to_cal = format_to_cal.rename(columns=new_cols).sort_index(axis=1)

# เผื่อ format_to_cal ว่าง (เช่นยังสร้างไม่ได้)
if format_to_cal is None or format_to_cal.shape[1] == 0:
    eval_by_day = input_data_converse[["Datetime", "I(t)"]].rename(columns={"Datetime":"Datetime"})
else:
    # จัดการหน่วยความจำ
    format_to_cal = format_to_cal.sort_index()
    format_to_cal = format_to_cal.astype("float32")

    # ใช้ actual_at_target ที่เตรียมไว้ด้านบน (index = Datetime_target)
    eval_by_day = format_to_cal.join(actual_at_target[["I(t)"]], how="outer")

    eval_by_day.index.name = "Datetime"
    eval_by_day = eval_by_day.reset_index()

# จัดลำดับคอลัมน์: "Datetime", "I(t)", แล้วตามด้วย yhat_from_origin_D-*
front = ["Datetime", "I(t)"]
rest  = [c for c in eval_by_day.columns if c not in front]
eval_by_day = eval_by_day[front + rest]

eval_by_day = eval_by_day.drop(eval_by_day.index[0]).reset_index(drop=True)
eval_wide = eval_wide.drop(eval_wide.index[0]).reset_index(drop=True)
eval_wide["Datetime"] = pd.to_datetime(eval_wide["Datetime"])
eval_by_day["Datetime"] = pd.to_datetime(eval_by_day["Datetime"])

def merge_actuals(input_df):
    merged = input_df.merge(
        e[["Datetime", "Irradiance (W/m2)_imputed"]],
        on="Datetime",
        how="left"
    )
    merged["I(t)"] = merged["Irradiance (W/m2)_imputed"]
    merged = merged.drop(columns="Irradiance (W/m2)_imputed")
    return merged


def rename_day_columns(input_df):
    for idx, col in enumerate(input_df.columns[2:], start=1):
        input_df.rename(columns={col: f"day{idx}"}, inplace=True)
    return input_df


def replace_negativevalues_with_zero(input_df):
    target_cols = ["day1", "day2", "day3"]
    for col in target_cols:
        mask = (input_df[col] < 1e-4) & input_df[col].notna()
        input_df.loc[mask, col] = 0
    return input_df

eval_wide = merge_actuals(eval_wide)
eval_by_day = merge_actuals(eval_by_day)
eval_by_day = rename_day_columns(eval_by_day)
eval_by_day = replace_negativevalues_with_zero(eval_by_day)




In [ ]:
import pandas as pd
import numpy as np

# convert for Load
STEP_MIN = 5  # 1 step = 5 นาที

# -------------------------
# LOAD
# -------------------------
input_data_converse = pd.read_csv("TimeGPTlag7dayLoad.csv", parse_dates=["Datetime"])
e = pd.read_csv("load_feature_5minresample_edited.csv")
e["Datetime"] = pd.to_datetime(e["Datetime"])

# -------------------------
# PARSE HORIZON COLUMNS
# -------------------------
column_frame = [c for c in input_data_converse.columns if c.startswith("Load_hat(t+")]
# map: column name -> k (เช่น "I_hat(t+73|t)" -> 73)
col2k = pd.Series(column_frame, index=column_frame).str.extract(r"Load_hat\(t\+(\d+)\|t\)")[0].astype(int).to_dict()

# -------------------------
# LONG TABLE: (origin -> target)
# -------------------------
long = input_data_converse.melt(
    id_vars=["Datetime", "Load(t)"], value_vars=column_frame,
    var_name="col", value_name="Ltot"
)
long["k"] = long["col"].map(col2k).astype(int)
long["Datetime_target"] = long["Datetime"] + pd.to_timedelta(long["k"] * STEP_MIN, unit="min")
long = long.rename(columns={"Datetime":"Datetime_origin", "Load(t)":"y_origin"})
long = long[["Datetime_origin", "Datetime_target", "k", "Ltot"]]

# -------------------------
# WIDE (by k): Ltot(t|t-k)  for k=1..864
# หมายเหตุ: ในเคส origin วันละครั้งที่ 23:55 ตารางนี้จะค่อนข้าง "sparse"
# -------------------------
wide_k = long.pivot_table(index="Datetime_target", columns="k", values="Ltot", aggfunc="last")
max_k = max(col2k.values())
# ให้มีครบทุกคอลัมน์ 1..max_k
wide_k = wide_k.reindex(columns=range(1, max_k+1))
wide_k.columns = [f"Ltot(t|t-{k})" for k in wide_k.columns]

# แนบค่าจริง y(t) ถ้ามีที่เวลา target (ถ้าไม่มี จะเป็น NaN)
# ---- ช่วยประหยัดหน่วยความจำ ----
wide_k = wide_k.sort_index()          # เรียงด้วย index (Datetime_target) แทน sort_values ทีหลัง
wide_k = wide_k.astype("float32")     # ลด dtype จาก float64 -> float32

# เปลี่ยนชื่อคอลัมน์ Ltot(...) -> Ihat(...)
wide_k.columns = [c.replace("Ltot(", "Load_hat(") for c in wide_k.columns]

# ค่าจริงที่เวลา target
actual_at_target = (
    input_data_converse[["Datetime", "Load(t)"]]
      .rename(columns={"Datetime": "Datetime_target"})
      .set_index("Datetime_target")
      .sort_index()
)
if actual_at_target["Load(t)"].dtype != "float32":
    actual_at_target["Load(t)"] = actual_at_target["Load(t)"].astype("float32")

# join แทน merge + sort_values (ประหยัด RAM มากกว่า)
eval_wide = wide_k.join(actual_at_target[["Load(t)"]], how="outer")

# รีเซ็ต index และจัดคอลัมน์ตามต้องการ
eval_wide.index.name = "Datetime"
eval_wide = eval_wide.reset_index()

# ให้มี Ihat(t|t-1)..Ihat(t|t-max_k) ครบและเรียง
max_k = max(col2k.values())
need_column = [f"Load_hat(t|t-{k})" for k in range(1, max_k+1)]
for name in need_column:
    if name not in eval_wide.columns:
        eval_wide[name] = np.nan

eval_wide = eval_wide[["Datetime", "Load(t)"] + need_column]


# -------------------------
# OPTIONAL: version แยกตาม "วันของ origin"
# (เหมาะมากสำหรับเคส forecast วันละครั้ง 23:55)
# จะได้คอลัมน์: yhat_from_origin_D-0, yhat_from_origin_D-1, yhat_from_origin_D-2, yhat_from_origin_D-3
# -------------------------
tmp = long.copy()
tmp["origin_day_offset"] = (tmp["Datetime_target"].dt.normalize() - tmp["Datetime_origin"].dt.normalize()).dt.days
format_to_cal = tmp.pivot_table(index="Datetime_target", columns="origin_day_offset", values="Ltot", aggfunc="last")

# จัดชื่อคอลัมน์ให้อ่านง่าย
if format_to_cal is not None and format_to_cal.shape[1] > 0:
    new_cols = {}
    for off in format_to_cal.columns:
        if off == 0:
            new_cols[off] = "yhat_from_origin_D-0"   # พยากรณ์ที่ทำเมื่อคืน 23:55 ของ "วันเดียวกัน"
        elif off == 1:
            new_cols[off] = "yhat_from_origin_D-1"   # ทำที่ 23:55 ของ "เมื่อวาน"
        elif off == 2:
            new_cols[off] = "yhat_from_origin_D-2"
        elif off == 3:
            new_cols[off] = "yhat_from_origin_D-3"
        else:
            new_cols[off] = f"yhat_from_origin_D-{off}"
    format_to_cal = format_to_cal.rename(columns=new_cols).sort_index(axis=1)

# เผื่อ format_to_cal ว่าง (เช่นยังสร้างไม่ได้)
if format_to_cal is None or format_to_cal.shape[1] == 0:
    eval_by_day = input_data_converse[["Datetime", "Load(t)"]].rename(columns={"Datetime":"Datetime"})
else:
    # จัดการหน่วยความจำ
    format_to_cal = format_to_cal.sort_index()
    format_to_cal = format_to_cal.astype("float32")

    # ใช้ actual_at_target ที่เตรียมไว้ด้านบน (index = Datetime_target)
    eval_by_day = format_to_cal.join(actual_at_target[["Load(t)"]], how="outer")

    eval_by_day.index.name = "Datetime"
    eval_by_day = eval_by_day.reset_index()

# จัดลำดับคอลัมน์: "Datetime", "I(t)", แล้วตามด้วย yhat_from_origin_D-*
front = ["Datetime", "Load(t)"]
rest  = [c for c in eval_by_day.columns if c not in front]
eval_by_day = eval_by_day[front + rest]

eval_by_day = eval_by_day.drop(eval_by_day.index[0]).reset_index(drop=True)
eval_wide = eval_wide.drop(eval_wide.index[0]).reset_index(drop=True)
eval_wide["Datetime"] = pd.to_datetime(eval_wide["Datetime"])
eval_by_day["Datetime"] = pd.to_datetime(eval_by_day["Datetime"])

def merge_actuals(input_df):
    merged = input_df.merge(
        e[["Datetime", "Ltot_impute (kW)"]],
        on="Datetime",
        how="left"
    )
    merged["Load(t)"] = merged["Ltot_impute (kW)"]
    merged = merged.drop(columns="Ltot_impute (kW)")
    return merged


def rename_day_columns(input_df):
    for idx, col in enumerate(input_df.columns[2:], start=1):
        input_df.rename(columns={col: f"day{idx}"}, inplace=True)
    return input_df


def replace_negativevalues_with_zero(input_df):
    target_cols = ["day1", "day2", "day3"]
    for col in target_cols:
        mask = (input_df[col] < 1e-4) & input_df[col].notna()
        input_df.loc[mask, col] = 0
    return input_df

eval_wide = merge_actuals(eval_wide)
eval_by_day = merge_actuals(eval_by_day)
eval_by_day = rename_day_columns(eval_by_day)
eval_by_day = replace_negativevalues_with_zero(eval_by_day)

In [ ]:
eval_by_day